In [1]:
!pip install langchain langchain-community psycopg2-binary sqlalchemy groq sqlalchemy langchain_groq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.5 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 14.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.1/326.1 kB 7.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.6/241.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.3/198.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.9/222.9 kB 6.9 MB/s 

In [ ]:
import os
os.environ["GROQ_API_KEY"] = ""

In [3]:
from sqlalchemy import create_engine, text
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_groq import ChatGroq
from langchain.agents.agent_types import AgentType

In [4]:
engine = create_engine("sqlite:///:memory:")

In [5]:
with engine.begin() as conn:
    # Create customers table
    conn.execute(text("""
        CREATE TABLE customers (
            id INTEGER PRIMARY KEY,
            name TEXT,
            email TEXT,
            location TEXT,
            signup_date DATE
        );
    """))

    conn.execute(text("""
        INSERT INTO customers (name, email, location, signup_date) VALUES
        ('Pradish', 'pradish@example.com', 'Chennai', '2022-01-15'),
        ('Ravi', 'ravi@example.com', 'Mumbai', '2023-03-10'),
        ('Anjali', 'anjali@example.com', 'Chennai', '2024-06-25'),
        ('Sara', 'sara@example.com', 'Delhi', '2023-11-02'),
        ('Kiran', 'kiran@example.com', 'Bangalore', '2021-12-30');
    """))

    # Create products table
    conn.execute(text("""
        CREATE TABLE products (
            id INTEGER PRIMARY KEY,
            name TEXT,
            category TEXT,
            price REAL
        );
    """))

    conn.execute(text("""
        INSERT INTO products (name, category, price) VALUES
        ('Laptop', 'Electronics', 60000),
        ('Headphones', 'Electronics', 1500),
        ('Coffee Mug', 'Home', 300),
        ('Yoga Mat', 'Fitness', 1200),
        ('Running Shoes', 'Fitness', 3500);
    """))

    # Create orders table
    conn.execute(text("""
        CREATE TABLE orders (
            id INTEGER PRIMARY KEY,
            customer_id INTEGER,
            product_id INTEGER,
            order_date DATE,
            quantity INTEGER,
            FOREIGN KEY(customer_id) REFERENCES customers(id),
            FOREIGN KEY(product_id) REFERENCES products(id)
        );
    """))

    conn.execute(text("""
        INSERT INTO orders (customer_id, product_id, order_date, quantity) VALUES
        (1, 1, '2024-12-01', 1),
        (1, 2, '2025-01-15', 2),
        (2, 3, '2025-02-20', 3),
        (3, 5, '2025-06-12', 1),
        (4, 4, '2025-07-01', 2),
        (5, 1, '2025-08-01', 1);
    """))

In [7]:
prompt = """ 
You are DBQueryBot, a helpful assistant designed to answer questions **strictly based on the contents of the connected SQL database**. The database contains three tables: `customers`, `products`, and `orders`.

⚠️ Rules:
- Only answer questions that can be answered using the database schema and data.
- If a question is out of scope or unrelated to the data, politely respond: "I'm only able to answer questions based on the database."
- Do not make assumptions, hallucinate values, or provide opinions.
- Always use the most efficient SQL query based on the schema.
- Format answers in simple, clear language.
- Include query results in the answer, if applicable.
- Your name is DBQueryBot. Never pretend to be a general-purpose assistant.

You may refer to:
- `customers`: id, name, email, location, signup_date
- `products`: id, name, category, price
- `orders`: id, customer_id, product_id, order_date, quantity

Start your response with a short answer and optionally include the query logic behind it if needed.

"""

In [ ]:
llm = ChatGroq(
    temperature=0,
    model_name="llama3-8b-8192",
    api_key=""
)



In [14]:
db = SQLDatabase(engine=engine)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [15]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [16]:
question = "Which customers from Chennai have purchased products in the Fitness category?"

response = agent_executor.run(question)
print(response)



> Entering new SQL Agent Executor chain...
Let's start by listing the tables in the database:

Action: sql_db_list_tables
Thought: Now that I have the list of tables, I can query the schema of the relevant tables to see what columns I can use in my query. I'm interested in customers from Chennai who have purchased products in the Fitness category, so I'll focus on the customers, orders, and products tables.

Action: sql_db_schema
Action Input: customers, orders, products
CREATE TABLE customers (
	id INTEGER, 
	name TEXT, 
	email TEXT, 
	location TEXT, 
	signup_date DATE, 
	PRIMARY KEY (id)
)

/*
3 rows from customers table:
id	name	email	location	signup_date
1	Pradish	pradish@example.com	Chennai	2022-01-15
2	Ravi	ravi@example.com	Mumbai	2023-03-10
3	Anjali	anjali@example.com	Chennai	2024-06-25
*/


CREATE TABLE orders (
	id INTEGER, 
	customer_id INTEGER, 
	product_id INTEGER, 
	order_date DATE, 
	quantity INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(customer_id) REFERENCES customers (

In [17]:
question = "Which customers has spent a lot of money"

response = agent_executor.run(question)
print(response)



> Entering new SQL Agent Executor chain...
Let's start by listing the tables in the database:

Thought: I should use sql_db_list_tables to get a list of tables in the database.
Action: sql_db_list_tables
Thought: Now that I have a list of tables, I should look at the schema of the tables to see what columns are available. I'm interested in finding out which customers have spent a lot of money, so I'll focus on the customers and orders tables.

Action: sql_db_schema
Action Input: customers, orders
CREATE TABLE customers (
	id INTEGER, 
	name TEXT, 
	email TEXT, 
	location TEXT, 
	signup_date DATE, 
	PRIMARY KEY (id)
)

/*
3 rows from customers table:
id	name	email	location	signup_date
1	Pradish	pradish@example.com	Chennai	2022-01-15
2	Ravi	ravi@example.com	Mumbai	2023-03-10
3	Anjali	anjali@example.com	Chennai	2024-06-25
*/


CREATE TABLE orders (
	id INTEGER, 
	customer_id INTEGER, 
	product_id INTEGER, 
	order_date DATE, 
	quantity INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(customer_

In [18]:
question = "tell me the top spender"

response = agent_executor.run(question)
print(response)



> Entering new SQL Agent Executor chain...
Let's start by listing the tables in the database:

Action: sql_db_list_tables
Thought: Now that I have the list of tables, I should query the schema of the tables to see what columns they have. This will help me construct a query to find the top spender.

Action: sql_db_schema
Action Input: customers, orders
CREATE TABLE customers (
	id INTEGER, 
	name TEXT, 
	email TEXT, 
	location TEXT, 
	signup_date DATE, 
	PRIMARY KEY (id)
)

/*
3 rows from customers table:
id	name	email	location	signup_date
1	Pradish	pradish@example.com	Chennai	2022-01-15
2	Ravi	ravi@example.com	Mumbai	2023-03-10
3	Anjali	anjali@example.com	Chennai	2024-06-25
*/


CREATE TABLE orders (
	id INTEGER, 
	customer_id INTEGER, 
	product_id INTEGER, 
	order_date DATE, 
	quantity INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(customer_id) REFERENCES customers (id), 
	FOREIGN KEY(product_id) REFERENCES products (id)
)

/*
3 rows from orders table:
id	customer_id	product_id	order_da